In [1]:
import numpy as np
import skimage
import os
from os import listdir
import matplotlib.pyplot as plt
import pandas as pd
import colorsys
from os.path import isfile, join
import pandas as pd
from PIL import Image
import colorsys
from skimage.measure import regionprops_table
import cv2

In [2]:
imagePath = '/Volumes/Aorta/aorta/preprocessedImages/cellSize/stitched/cropped1-8/nuclei'
segPath = '/Volumes/Aorta/aorta/segmentations/cellSize/stitchCyto/'
cropPath = '/Volumes/Aorta/aorta/preprocessedImages/cellSize/croppedNew/cropped1-8/nuclei'
savePath = '/Volumes/Aorta/aorta/results/cellSizeStatistics/cellsAlongAxis'
dfOrientation = pd.read_csv('/Users/jones/Downloads/orientationNucleiImages.csv')
dfCellStat = pd.read_csv('/Volumes/Aorta/aorta/results/cellSizeStatistics/allCellShapesUpdate_MicroCTAspect.csv')
pixelsWith = 500

In [112]:
dfCellAorta = dfCellStat.groupby(['age', 'aorta']).median()
dfCellAorta = dfCellAorta.reset_index()

In [113]:
dfCellAorta = dfCellAorta.reset_index()

In [5]:
###connect segmentation

def connectSegmentation(segPath, maskImage, cropPath):
    shapes = []
    cropping_parameter_width = int(maskImage.shape[1]/550)
    cropping_parameter_height = int(maskImage.shape[0]/550)
    k=1
    start = 0
    for i in range(cropping_parameter_height):
        for j in range(cropping_parameter_width):
            maskFile = "Cropped_IMG-" + str(k) + "_seg.npy"
            if os.path.exists(join(segPath, "Cropped_IMG-" + str(k) + "_seg.npy")):
                input= np.load(join(segPath, "Cropped_IMG-" + str(k) + "_seg.npy"), allow_pickle=True).item()['masks']
                outlines = np.load(join(segPath, "Cropped_IMG-" + str(k) + "_seg.npy"), allow_pickle=True).item()['outlines']
                unique_labels = np.unique(input)
                new_labels = np.concatenate([[0],np.arange(start, start + len(unique_labels)-1)])
                label_mapping = dict(zip(unique_labels, new_labels))
                input = np.vectorize(label_mapping.get)(input)
                outlines = np.vectorize(label_mapping.get)(outlines)
                start = np.max(new_labels) +1
            else:
                imageHelp = skimage.io.imread(join(cropPath, "Cropped_IMG-" + str(k) + ".tif"))
                input = np.zeros(imageHelp.shape)
                outlines = np.zeros(imageHelp.shape)
            shapes.append(input.shape)

            if j == 0:
                if i == 0:
                    concatenated = input
                    concatenatedOutlines = outlines
                else:
                    concatenatedHelp = input
                    concatenatedOutlinesHelp = outlines
            else:
                masks = input
                outline = outlines
                if i == 0:
                    concatenated = np.concatenate((concatenated, masks), axis = 1)
                    concatenatedOutlines = np.concatenate((concatenatedOutlines, outline), axis = 1)
                else:
                    concatenatedHelp = np.concatenate((concatenatedHelp, masks), axis = 1)
                    concatenatedOutlinesHelp = np.concatenate((concatenatedOutlinesHelp, outline), axis = 1)
            k +=1

        if i !=0:
            concatenated = np.concatenate((concatenated, concatenatedHelp), axis = 0)
            concatenatedOutlines = np.concatenate((concatenatedOutlines, concatenatedOutlinesHelp), axis = 0)
    
    return concatenated, concatenatedOutlines, shapes

def connectSegmentationOnlyMasks(segPath, maskImage, cropPath):
    cropping_parameter_width = int(maskImage.shape[1]/550)
    cropping_parameter_height = int(maskImage.shape[0]/550)
    k=1
    start = 0
    for i in range(cropping_parameter_height):
        for j in range(cropping_parameter_width):
            maskFile = "Cropped_IMG-" + str(k) + "_seg.npy"
            if os.path.exists(join(segPath, "Cropped_IMG-" + str(k) + "_seg.npy")):
                input= np.load(join(segPath, "Cropped_IMG-" + str(k) + "_seg.npy"), allow_pickle=True).item()['masks']
                unique_labels = np.unique(input)
                new_labels = np.concatenate([[0],np.arange(start, start + len(unique_labels)-1)])
                label_mapping = dict(zip(unique_labels, new_labels))
                input = np.vectorize(label_mapping.get)(input)
                start = np.max(new_labels) +1
            else:
                imageHelp = skimage.io.imread(join(cropPath, "Cropped_IMG-" + str(k) + ".tif"))
                input = np.zeros(imageHelp.shape)

            if j == 0:
                if i == 0:
                    concatenated = input
                else:
                    concatenatedHelp = input
            else:
                masks = input
                if i == 0:
                    concatenated = np.concatenate((concatenated, masks), axis = 1)
                else:
                    concatenatedHelp = np.concatenate((concatenatedHelp, masks), axis = 1)
            k +=1

        if i !=0:
            concatenated = np.concatenate((concatenated, concatenatedHelp), axis = 0)
    
    return concatenated


def plotMasks(img, masks):
    img = Image.fromarray(img)
    imgGray = img.convert('L')
    imgGrayarray = np.asarray(imgGray)
    rgb = mask_overlay(imgGrayarray, masks, colors=None)
    fig, ax = plt.subplots(figsize=(36,15))
    ax.imshow(rgb)

def mask_overlay(img, masks, colors=None):

    img = img.astype(np.float32)

    HSV = np.zeros((img.shape[0], img.shape[1], 3), np.float32)
    HSV[:,:,2] = np.clip((img / 255. if img.max() > 1 else img) * 1.5, 0, 1)
    hues = np.linspace(0, 1, masks.max()+1)[np.random.permutation(masks.max())]
    for n in range(int(masks.max())):
        ipix = (masks==n+1).nonzero()
        if colors is None:
            HSV[ipix[0],ipix[1],0] = hues[n]
        else:
            HSV[ipix[0],ipix[1],0] = colors[n,0]
        HSV[ipix[0],ipix[1],1] = 1.0
    RGB = (hsv_to_rgb(HSV) * 255).astype(np.uint8)
    return RGB


def hsv_to_rgb(arr):
    hsv_to_rgb_channels = np.vectorize(colorsys.hsv_to_rgb)
    h, s, v = np.rollaxis(arr, axis=-1)
    r, g, b = hsv_to_rgb_channels(h, s, v)
    rgb = np.stack((r,g,b), axis=-1)
    return rgb


def getNucs(segFullImage, accuracy, axisLength, type, angle, delta_x, imageAdjusted):
    if type == "length":
        t = 0
    elif type == "circumference":
        t = 1

    prop_dict = regionprops_table(segFullImage, properties = ['label','area'])
    df = pd.DataFrame(prop_dict)
    segBinary = np.where(segFullImage != 0, 1, 0)

    nucs = []
    densities =[]
    segBinary = np.where(segFullImage != 0, 1, 0)
    start_left = np.min(np.where(segBinary[:,:100] == 1)[t])
    end_left = np.int(start_left + axisLength)
    delta_y = delta_x * np.tan(np.radians(angle))
    while(end_left < np.max(np.where(segBinary[:,:100] == 1)[t])):
        start_right = np.int(start_left + delta_y)
        end_right = np.int(start_right + axisLength)
        rectangleImage = np.zeros(segBinary.shape, dtype=np.uint8)
        pointsRectangle = np.array([(0, start_left), (segFullImage.shape[1],start_right), (segFullImage.shape[1], end_right), (0, end_left)])
        cv2.fillPoly(rectangleImage, pts= [pointsRectangle], color= 255)
        tiled = segFullImage[np.where(rectangleImage == 255)]
        tiledImage = imageAdjusted[np.where(rectangleImage == 255)]
        tiled= tiled.astype(int)
        tiledImage = tiledImage.astype(int)
        unique, counts = np.unique(tiled, return_counts=True)
        df_help = pd.DataFrame({'label': unique[unique!=0], 'area': counts[unique!=0]})
        merged_df = pd.merge(df_help, df, on='label', suffixes=('_tile', '_all'))
        merged_df['fraction'] = merged_df['area_tile']/merged_df['area_all']
        nuc = len(merged_df[merged_df.fraction > accuracy])
        area = len(np.where(tiledImage==255)[0])
        if area > 0:
            nucs.append(nuc)
            densities.append(nuc/area)
        start_left = start_left + np.int(axisLength/2)
        end_left = np.int(start_left + axisLength)
        
    


    end_left = np.max(np.where(segBinary[:,:100] == 1)[t])
    start_left = end_left - axisLength
    start_right = np.int(start_left + delta_y)
    end_right = np.int(start_right + axisLength)
    rectangleImage = np.zeros(segBinary.shape, dtype=np.uint8)
    pointsRectangle = np.array([(0, start_left), (segFullImage.shape[1],start_right), (segFullImage.shape[1], end_right), (0, end_left)])
    cv2.fillPoly(rectangleImage, pts= [pointsRectangle], color= 255)
    tiled = segFullImage[np.where(rectangleImage == 255)]
    tiledImage = imageAdjusted[np.where(rectangleImage == 255)]
    tiled= tiled.astype(int)
    tiledImage = tiledImage.astype(int)
    unique, counts = np.unique(tiled, return_counts=True)
    df_help = pd.DataFrame({'label': unique[unique!=0], 'area': counts[unique!=0]})
    merged_df = pd.merge(df_help, df, on='label', suffixes=('_tile', '_all'))
    merged_df['fraction'] = merged_df['area_tile']/merged_df['area_all']
    area = len(np.where(tiledImage==255)[0])
    if area > 0:
            nucs.append(nuc)
            densities.append(nuc/area)

    nucs = np.asarray(nucs)
    densities = np.asarray(densities)

    return nucs, densities



def getNucsCircum(segFullImage, accuracy, axisLength, angle, delta_y, imageAdjusted):

    prop_dict = regionprops_table(segFullImage, properties = ['label','area'])
    df = pd.DataFrame(prop_dict)
    segBinary = np.where(segFullImage != 0, 1, 0)

    nucs = []
    densities =[]
    segBinary = np.where(segFullImage != 0, 1, 0)



    start_upper = 0
    end_upper = start_upper + axisLength
    delta_x = delta_y * np.tan(np.radians(angle))
    while(end_upper < maskImage.shape[1]):
        start_down = np.int(start_upper + delta_x)
        end_down = np.int(start_down + axisLength)
        rectangleImage = np.zeros(segBinary.shape, dtype=np.uint8)
        pointsRectangle = np.array([(start_upper,0), (start_down, segFullImage.shape[0]), (end_down, segFullImage.shape[0]), (end_upper,0)])
        cv2.fillPoly(rectangleImage, pts= [pointsRectangle], color= 255)
        tiled = segFullImage[np.where(rectangleImage == 255)]
        tiledImage = imageAdjusted[np.where(rectangleImage == 255)]
        tiled= tiled.astype(int)
        tiledImage = tiledImage.astype(int)
        unique, counts = np.unique(tiled, return_counts=True)
        df_help = pd.DataFrame({'label': unique[unique!=0], 'area': counts[unique!=0]})
        merged_df = pd.merge(df_help, df, on='label', suffixes=('_tile', '_all'))
        merged_df['fraction'] = merged_df['area_tile']/merged_df['area_all']
        nuc = len(merged_df[merged_df.fraction > accuracy])
        area = len(np.where(tiledImage==255)[0])
        if area > 0:
            nucs.append(nuc)
            densities.append(nuc/area)
        start_upper = start_upper + np.int(axisLength/2)
        end_upper = np.int(start_upper + axisLength)
        
    


    end_upper = maskImage.shape[1]
    start_upper = end_upper - axisLength

    start_down = np.int(start_upper + delta_x)
    end_down = np.int(start_down + axisLength)
    rectangleImage = np.zeros(segBinary.shape, dtype=np.uint8)
    pointsRectangle = np.array([(start_upper,0), (start_down, segFullImage.shape[0]), (end_down, segFullImage.shape[0]), (end_upper,0)])
    cv2.fillPoly(rectangleImage, pts= [pointsRectangle], color= 255)
    tiled = segFullImage[np.where(rectangleImage == 255)]
    tiledImage = imageAdjusted[np.where(rectangleImage == 255)]
    tiled= tiled.astype(int)
    tiledImage = tiledImage.astype(int)
    unique, counts = np.unique(tiled, return_counts=True)
    df_help = pd.DataFrame({'label': unique[unique!=0], 'area': counts[unique!=0]})
    merged_df = pd.merge(df_help, df, on='label', suffixes=('_tile', '_all'))
    merged_df['fraction'] = merged_df['area_tile']/merged_df['area_all']
    area = len(np.where(tiledImage==255)[0])
    if area > 0:
            nucs.append(nuc)
            densities.append(nuc/area)

    nucs = np.asarray(nucs)
    densities = np.asarray(densities)

    return nucs, densities

In [ ]:
##nuc along circum

df = pd.DataFrame()
dfOverview = pd.DataFrame()
folders = [x[0] for x in os.walk(imagePath)]
folders = sorted(folders)

for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if "Mask.tif" in f if "._" not in f]
    for file in files:
        age = folder.split("/")[-2]
        aorta = folder.split("/")[-1]
        ageInt = int(age.split("P")[1])
        pixelsWith = int(np.round(dfCellAorta.loc[(dfCellAorta['age'] == ageInt) & (dfCellAorta['aorta'] == aorta)]['major_axis_length'].values[0]))
        maskImage = skimage.io.imread(join(folder, file))
        negFile = file.split('.tif')[0] + 'Neg' + file.split('Mask')[1]
        maskImageNeg = skimage.io.imread(join(folder, negFile))
        imageAdjusted = np.where(maskImageNeg==0, 0, maskImage)
        segFilesPath = join(segPath, age, aorta)
        croppingPath = join(cropPath, age, aorta)
        segFullImage = connectSegmentationOnlyMasks(segFilesPath, maskImage, croppingPath)
        segFullImage = segFullImage.astype('int')
        dfFilter = dfOrientation.loc[(dfOrientation['age'] == age) & (dfOrientation['aorta'] == aorta)]
        angle = -dfFilter['Angle'].mean()
        nucsLength, densities = getNucsCircum(segFullImage, 0.75, pixelsWith, angle, segFullImage.shape[0], imageAdjusted)
        
        df_help = pd.DataFrame({'age': age, 'aorta': aorta, 'nucs': nucsLength, 'densities':densities })
        saveName = "aortaCircumferenceAxis_" + age + "_" + aorta + "Mean.csv"
        df_help.to_csv(join(savePath, saveName), index = False)
        df = pd.concat([df, df_help])
        
df.to_csv('/Users/jones/Downloads/nucsAlongCircum.csv', index = False)

##nuc along length
df = pd.DataFrame()
folders = [x[0] for x in os.walk(imagePath)]
folders = sorted(folders)

for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if "Mask.tif" in f if "._" not in f]
    for file in files:
        age = folder.split("/")[-2]
        aorta = folder.split("/")[-1]
        ageInt = int(age.split("P")[1])
        pixelsWith = int(np.round(dfCellAorta.loc[(dfCellAorta['age'] == ageInt) & (dfCellAorta['aorta'] == aorta)]['minor_axis_length'].values[0]))
        maskImage = skimage.io.imread(join(folder, file))
        negFile = file.split('.tif')[0] + 'Neg' + file.split('Mask')[1]
        maskImageNeg = skimage.io.imread(join(folder, negFile))
        imageAdjusted = np.where(maskImageNeg==0, 0, maskImage)
        segFilesPath = join(segPath, age, aorta)
        croppingPath = join(cropPath, age, aorta)
        segFullImage = connectSegmentationOnlyMasks(segFilesPath, maskImage, croppingPath)
        segFullImage = segFullImage.astype('int')
        dfFilter = dfOrientation.loc[(dfOrientation['age'] == age) & (dfOrientation['aorta'] == aorta)]
        angle = -dfFilter['Angle'].mean()
        nucsLength, densities = getNucs(segFullImage, 0.75, pixelsWith , "length",angle, segFullImage.shape[1], imageAdjusted)
        
        df_help = pd.DataFrame({'age': age, 'aorta': aorta, 'nucs': nucsLength, 'densities':densities })
        saveName = "aortaLengthAxis_" + age + "_" + aorta + "Mean.csv"
        df_help.to_csv(join(savePath, saveName), index = False)
        df = pd.concat([df, df_help])

df.to_csv('/Users/jones/Downloads/nucsAlongLength.csv', index = False)
